In [3]:
import numpy as np
import random

# Define the grid
grid = [
    ['S', ' ', ' ', 'X', ' '],
    [' ', 'X', ' ', ' ', ' '],
    [' ', ' ', ' ', 'X', ' '],
    ['X', ' ', ' ', ' ', ' '],
    [' ', ' ', 'X', ' ', 'G']
]

# Define actions
actions = ['up', 'down', 'left', 'right']

# Q-table: 3D array to store Q-values for each state-action pair
# Dimensions: 5 (rows) x 5 (columns) x 4 (actions)
q_table = np.zeros((5, 5, 4))

# Hyperparameters

# Learning rate (alpha): Determines how much new information overrides old information
# Range: 0.0 to 1.0
# - Low value: Slower learning, but more stable
# - High value: Faster learning, but may oscillate or fail to converge
# 0.1 is a common starting point, balancing learning speed and stability
alpha = 0.1

# Discount factor (gamma): Determines the importance of future rewards
# Range: 0.0 to 1.0
# - Low value: Agent focuses on immediate rewards
# - High value: Agent considers long-term rewards more strongly
# 0.9 is often used, encouraging the agent to consider long-term consequences while still prioritizing nearer rewards
gamma = 0.9

# Exploration rate (epsilon): Controls the trade-off between exploration and exploitation
# Range: 0.0 to 1.0
# - Low value: Agent exploits known information more (greedy behavior)
# - High value: Agent explores new actions more frequently
# 0.1 means the agent will explore random actions 10% of the time, and choose the best known action 90% of the time
# This value is often decreased over time to reduce exploration as the agent learns
epsilon = 0.1


# Function to get next state
def get_next_state(state, action):
    """
    Determines the next state given the current state and action.
    
    This function implements the transition model of the environment, 
    defining how the agent moves in the grid world.
    
    Parameters:
    - state: A tuple (x, y) representing the current position in the grid
    - action: A string representing the chosen action ('up', 'down', 'left', 'right')
    
    Returns:
    - A tuple (x, y) representing the next state
    
    Note:
    - The function checks for grid boundaries and obstacles ('X')
    - If the move is invalid (hits a wall or obstacle), the agent stays in the current state
    """
    x, y = state
    if action == 'up' and x > 0 and grid[x-1][y] != 'X':
        return (x-1, y)
    elif action == 'down' and x < 4 and grid[x+1][y] != 'X':
        return (x+1, y)
    elif action == 'left' and y > 0 and grid[x][y-1] != 'X':
        return (x, y-1)
    elif action == 'right' and y < 4 and grid[x][y+1] != 'X':
        return (x, y+1)
    return state

# Function to get reward
def get_reward(state):
    """
    Determines the reward for the given state.
    
    This function implements the reward model of the environment,
    defining the immediate feedback the agent receives for being in a particular state.
    
    Parameters:
    - state: A tuple (x, y) representing the position in the grid
    
    Returns:
    - A numerical value representing the reward
    
    Reward structure:
    - Reaching the goal (G): +10 (highest reward to encourage finding the goal)
    - Hitting an obstacle (X): -1 (penalty to discourage running into obstacles)
    - Any other move: -0.1 (small penalty to encourage finding the goal quickly)
    
    Note: This reward structure encourages the agent to find the goal
    while avoiding obstacles and minimizing the number of steps.
    """
    x, y = state
    if grid[x][y] == 'G':
        return 10
    elif grid[x][y] == 'X':
        return -1
    else:
        return -0.1

# Training loop
num_episodes = 1000

# Outer loop: Repeat the learning process for a number of episodes
for episode in range(num_episodes):
    # Initialize the starting state for each episode
    state = (0, 0)  # Start state (top-left corner)
    
    # Inner loop: Continue taking actions until the goal state is reached
    while grid[state[0]][state[1]] != 'G':
        # Choose action using epsilon-greedy strategy
        if random.uniform(0, 1) < epsilon:
            # Exploration: Choose a random action
            action = random.choice(actions)
        else:
            # Exploitation: Choose the action with the highest Q-value
            action = actions[np.argmax(q_table[state[0]][state[1]])]
        
        # Take the chosen action and observe the result
        next_state = get_next_state(state, action)
        reward = get_reward(next_state)
        
        # Update Q-table using the Q-learning formula
        # Q(s,a) = (1 - α) * Q(s,a) + α * (r + γ * max(Q(s',a')))
        current_q = q_table[state[0]][state[1]][actions.index(action)]
        max_future_q = np.max(q_table[next_state[0]][next_state[1]])
        new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)
        q_table[state[0]][state[1]][actions.index(action)] = new_q
        
        # Move to the next state
        state = next_state

# Function to find optimal path
def find_optimal_path():
    state = (0, 0)
    path = [state]
    
    while grid[state[0]][state[1]] != 'G':
        action = actions[np.argmax(q_table[state[0]][state[1]])]
        state = get_next_state(state, action)
        path.append(state)
    
    return path

# Print optimal path
optimal_path = find_optimal_path()
print("Optimal path:", optimal_path)

# Visualize the path
for i in range(5):
    for j in range(5):
        if (i, j) in optimal_path:
            print('O', end=' ')
        else:
            print(grid[i][j], end=' ')
    print()

Optimal path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (3, 4), (4, 4)]
O     X   
O X       
O O O X   
X   O O O 
    X   O 
